In [1]:
import glob
import os
import sys
import tqdm
import time
import numpy as np
import pickle

from yacos.essential import Engine, IO
from yacos.info import compy
from yacos.info.compy.extractors import LLVMDriver

In [2]:
# Instantiate the LLVM driver.
driver = LLVMDriver()
# Define the builder
builder = compy.LLVMGraphBuilder(driver)

In [3]:
visitors = {
    # Clang
    'ast': compy.ASTVisitor,
    'astdata': compy.ASTDataVisitor,
    'astdatacfg': compy.ASTDataCFGVisitor,
    # LLVM
    'programl': compy.LLVMProGraMLVisitor,
    'programlnoroot': compy.LLVMProGraMLNoRootVisitor,
    'cfg': compy.LLVMCFGVisitor,
    'cfgcompact': compy.LLVMCFGCompactVisitor,
    'cfgcall': compy.LLVMCFGCallVisitor,
    'cfgcallnoroot': compy.LLVMCFGCallNoRootVisitor,
    'cfgcallcompact': compy.LLVMCFGCallCompactVisitor,
    'cfgcallcompact1e': compy.LLVMCFGCallCompactOneEdgeVisitor,
    'cfgcallcompactnoroot': compy.LLVMCFGCallCompactNoRootVisitor,
    'cfgcallcompact1enoroot': compy.LLVMCFGCallCompactOneEdgeNoRootVisitor,
    'cdfg': compy.LLVMCDFGVisitor,
    'cdfgcompact': compy.LLVMCDFGCompactVisitor,
    'cdfgcompact1e': compy.LLVMCDFGCompactOneEdgeVisitor,
    'cdfgcall': compy.LLVMCDFGCallVisitor,
    'cdfgcallnoroot': compy.LLVMCDFGCallNoRootVisitor,
    'cdfgcallcompact': compy.LLVMCDFGCallCompactVisitor,
    'cdfgcallcompact1e': compy.LLVMCDFGCallCompactOneEdgeVisitor,
    'cdfgcallcompactnoroot': compy.LLVMCDFGCallCompactNoRootVisitor,
    'cdfgcallcompact1enoroot': compy.LLVMCDFGCallCompactOneEdgeNoRootVisitor,
    'cdfgplus': compy.LLVMCDFGPlusVisitor,
    'cdfgplusnoroot': compy.LLVMCDFGPlusNoRootVisitor
}

In [4]:
dataset_dir = '/home/nonroot/experiment/datasets/classify_seqs/group1.1000/'
folders = glob.glob(f"{dataset_dir}/*/*/")
print(f"There are {len(folders)} directories")

There are 5000 directories


In [5]:
results_dir = '/home/nonroot/experiment/results/notebook/'
os.makedirs(results_dir, exist_ok=True)

In [6]:
def extract_graph_data(graph, graph_type):
    """Convert the graph to StellarGraph representation."""

    nodes = {}

    if 'ast' in graph_type:
        nodes['w2v'] = graph.get_nodes_word2vec_embeddings('ast')
        nodes['boo'] = graph.get_nodes_bag_of_words_embeddings('ast')
    elif 'asm' in graph_type:
        nodes['boo'] = graph.get_nodes_bag_of_words_embeddings('ir')
    else:
        nodes['w2v'] = graph.get_nodes_word2vec_embeddings('ir')
        nodes['boo'] = graph.get_nodes_bag_of_words_embeddings('ir')
        nodes['i2v'] = graph.get_nodes_inst2vec_embeddings()
        nodes['ir2v'] = graph.get_nodes_ir2vec_embeddings()

    edges = graph.get_edges_dataFrame()

    return edges, nodes

In [7]:
optimizations = {
    '500-bSum':  '-mem2reg -jump-threading -instcombine -early-cse-memssa -jump-threading -licm -early-cse-memssa -sroa -simplifycfg -reassociate -instcombine -slp-vectorizer -early-cse-memssa',
    '4372-bMed': '-mem2reg -early-cse-memssa -correlated-propagation -instcombine -reassociate -simplifycfg -early-cse-memssa -instcombine -licm -jump-threading -simplifycfg -dse -reassociate -early-cse-memssa -instcombine',
    '5624-bLim': '-sroa -early-cse-memssa -reassociate -instcombine -simplifycfg -licm -speculative-execution -jump-threading -early-cse-memssa -simplifycfg -instcombine -simplifycfg',
    '6310-bGeo': '-loop-vectorize -sroa -gvn -instcombine -simplifycfg -instcombine -licm -gvn -correlated-propagation -jump-threading -mldst-motion -early-cse-memssa -instcombine -simplifycfg -instsimplify',
    '4211-bCap': '-loop-rotate -sroa -correlated-propagation -indvars -gvn -tailcallelim -instcombine -jump-threading -reassociate -simplifycfg -instcombine -early-cse-memssa',
    'O0':        '-O0', 
    'Os':        '-Os', 
    'Oz':        '-Oz'
}

representations = [
    'cfgcompact',
    #'cfgcallcompact',
    # 'cfgcallcompact1e',
    'cfgcallcompactnoroot',
    'cfgcallcompact1enoroot',
   # 'cdfgcompact',
   # 'cdfgcompact1e',
   # 'cdfgcallcompact',
   # 'cdfgcallcompact1e',
   # 'cdfgcallcompactnoroot',
   # 'cdfgcallcompact1enoroot'
]

start = time.time()
for optname, opt in optimizations.items():
    print(f"Starting with optmization {optname} with following passes: {opt}")
    bar = tqdm.tqdm(folders, desc="Processing...")
    for benchdir in bar:
        outfilename = benchdir[len(dataset_dir):-1].replace('/', '.')
        bar.set_description(outfilename)
        bar.refresh()
        try:
            Engine.cleanup(benchdir, 'opt')
            Engine.compile(benchdir, 'opt', opt)
            binsize = IO.load_yaml(os.path.join(benchdir, 'binary_size.yaml'))
            codesize = IO.load_yaml(os.path.join(benchdir, 'code_size.yaml'))
            compiletime = IO.load_yaml(os.path.join(benchdir, 'compile_time.yaml'))
        except BaseException as e:
            print(f"Error compiling benchmark at {benchdir} with {optname}: {e}")
            continue
            
        try:
            source = f'{benchdir}/a.out_o.bc'
            extinfo = builder.ir_to_info(source)
        except BaseException as e:
            print(f"Error extracting IR from {source}: {e}")
            continue
            
        for rep in representations:
            try:
                graph = builder.info_to_representation(extinfo, visitors[rep])
                edges, nodes = extract_graph_data(graph, rep)
                representation_dir = os.path.join(results_dir, optname, rep)
                os.makedirs(representation_dir, exist_ok=True)
                for extname, extdata in nodes.items():
                    d = dict()
                    d.update(binsize)
                    d.update(codesize)
                    d.update(compiletime)
                    outputfile = os.path.join(representation_dir, f"{outfilename}_{extname}")     
                    np.savez_compressed(outputfile, edges=edges, nodes=extdata, labels=d)
            except BaseException as e:
                print(f"Error extracting representation {rep} for {benchdir}: {e}")
                continue
end = time.time()
print(f"It took {end-start} seconds...")

Starting with optmization 500-bSum with following passes: -mem2reg -jump-threading -instcombine -early-cse-memssa -jump-threading -licm -early-cse-memssa -sroa -simplifycfg -reassociate -instcombine -slp-vectorizer -early-cse-memssa


SUM.extr_addi-datahwdrv_apci2200.c_i_APCI2200_WriteDigitalOutput:   0%|                                                                                                                                  | 0/5000 [00:00<?, ?it/s]/home/nonroot/.local/lib/python3.8/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
SUM.extr_book3s_64_mmu_hv.c_kvmppc_book3s_hv_page_fault:   2%|██▍                                                                                                                             | 95/5000 [02:28<2:18:57,  1.70s/it]

Error extracting representation cfgcompact for /home/nonroot/experiment/datasets/classify_seqs/group1.1000/SUM/extr_book3s_64_mmu_hv.c_kvmppc_book3s_hv_page_fault/: list index out of range
Error extracting representation cfgcallcompactnoroot for /home/nonroot/experiment/datasets/classify_seqs/group1.1000/SUM/extr_book3s_64_mmu_hv.c_kvmppc_book3s_hv_page_fault/: list index out of range


SUM.extr_tscParseInsert.c_tscParseSqlForCreateTableOnDemand:   2%|██▍                                                                                                                         | 96/5000 [02:30<2:09:32,  1.59s/it]

Error extracting representation cfgcallcompact1enoroot for /home/nonroot/experiment/datasets/classify_seqs/group1.1000/SUM/extr_book3s_64_mmu_hv.c_kvmppc_book3s_hv_page_fault/: list index out of range


...

Error extracting representation cfgcallcompact1enoroot for /home/nonroot/experiment/datasets/classify_seqs/group1.1000/BST/extr_gdb-stub.c_gdbstub_set_breakpoint/: list index out of range


BST.extr_s3fb.c_s3fb_set_par:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 4960/5000 [3:25:34<01:19,  1.98s/it]

Error extracting IR from /home/nonroot/experiment/datasets/classify_seqs/group1.1000/BST/extr_yuvjpeg.c_main//a.out_o.bc: Failed parse IR file


BST.extr_dragon4.c_FormatPositional: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [3:27:16<00:00,  2.49s/it]

It took 83395.8209195137 seconds...


In [5]:
!ls 4211-bCap/cfgcompact | head

BST.extr_1465.c_main_boo.npz
BST.extr_1465.c_main_i2v.npz
BST.extr_1465.c_main_ir2v.npz
BST.extr_1465.c_main_w2v.npz
BST.extr_19160.c_main_boo.npz
BST.extr_19160.c_main_i2v.npz
BST.extr_19160.c_main_ir2v.npz
BST.extr_19160.c_main_w2v.npz
BST.extr_20813.c_main_boo.npz
BST.extr_20813.c_main_i2v.npz
ls: write error: Broken pipe
